In [1]:
import pandas as pd  #untuk manipulasi dan analisis data.
import numpy as np #untuk komputasi numerik dengan array dan matriks.
from sklearn.model_selection import train_test_split #untuk membagi dataset menjadi data latih dan data uji.
from sklearn.preprocessing import StandardScaler #untuk melakukan preprocessing data, seperti standarisasi fitur.
from sklearn.metrics import accuracy_score #untuk menghitung akurasi model klasifikasi.
import torch #untuk membangun dan melatih model neural network.
import torch.nn as nn #Modul PyTorch yang berisi berbagai lapisan dan fungsi untuk membangun neural network.
import torch.optim as optim #Modul PyTorch yang berisi berbagai algoritma optimasi untuk melatih model neural network.
from torch.utils.data import DataLoader, TensorDataset #untuk membuat dan mengelola dataset, serta memuat data secara efisien selama pelatihan.

In [2]:
df = pd.read_csv('/content/heart.csv')
df.head()
#memuat dataset

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,52,1,0,125,212,0,1,168,0,1.0,2,2,3,0
1,53,1,0,140,203,1,0,155,1,3.1,0,0,3,0
2,70,1,0,145,174,0,1,125,1,2.6,0,0,3,0
3,61,1,0,148,203,0,1,161,0,0.0,2,1,3,0
4,62,0,0,138,294,1,1,106,0,1.9,1,3,2,0


In [3]:
X = df.drop('target', axis=1).values
y = df['target'].values
#memisahkan fitur dan target

In [4]:
scaler = StandardScaler()
X = scaler.fit_transform(X)
#normalisasi data

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
#split data menjadi training dan testing

In [5]:
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)
#mengonversi data ke dalam bentuk tensor

In [6]:
def create_mlp(input_size, hidden_layers, hidden_neurons, activation_function):
    layers = []
    layers.append(nn.Linear(input_size, hidden_neurons))
    #lapisan input ke lapisan tersembunyi pertama

    for _ in range(hidden_layers - 1):
        layers.append(nn.Linear(hidden_neurons, hidden_neurons))
    #menambahkan lapisan tersembunyi tambahan

    if activation_function == 'linear':
        activation = nn.Identity()
    elif activation_function == 'Sigmoid':
        activation = nn.Sigmoid()
    elif activation_function == 'ReLU':
        activation = nn.ReLU()
    elif activation_function == 'Softmax':
        activation = nn.Softmax(dim=1)
    elif activation_function == 'Tanh':
        activation = nn.Tanh()
    #memilih fungsi aktivasi

    layers.append(nn.Linear(hidden_neurons, 2))
    #menambahkan lapisan output

def create_mlp(input_size, hidden_layers, hidden_neurons, activation_function):
    layers = []
    layers.append(nn.Linear(input_size, hidden_neurons))

    for _ in range(hidden_layers - 1):
        layers.append(nn.Linear(hidden_neurons, hidden_neurons))
    #menambahkan lapisan tersembunyi tambahan

    if activation_function == 'linear':
        activation = nn.Identity()
    elif activation_function == 'Sigmoid':
        activation = nn.Sigmoid()
    elif activation_function == 'ReLU':
        activation = nn.ReLU()
    elif activation_function == 'Softmax':
        activation = nn.Softmax(dim=1)
    elif activation_function == 'Tanh':
        activation = nn.Tanh()
    #memilih fungsi aktivasi

    layers.append(nn.Linear(hidden_neurons, 2))
    #menambahkan lapisan output

    model = nn.Sequential(*layers)
    return model
    #menggabungkan semua lapisan menjadi satu model

#membuat model MLP

In [7]:
def train_and_evaluate(model, X_train_tensor, y_train_tensor, X_test_tensor, y_test_tensor, batch_size, epochs, learning_rate):
    train_data = TensorDataset(X_train_tensor, y_train_tensor)
    train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
    #dataLoader untuk pelatihan dalam batch

    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    criterion = nn.CrossEntropyLoss()
    #optimizer (Adam) dan fungsi loss (CrossEntropyLoss untuk klasifikasi)

    model.train()
    for epoch in range(epochs):
        for X_batch, y_batch in train_loader:
            optimizer.zero_grad()
            output = model(X_batch)
            loss = criterion(output, y_batch)
            loss.backward()
            optimizer.step()
    #loop pelatihan

    model.eval()
    with torch.no_grad():
    #evaluasi model setelah pelatihan
        train_output = model(X_train_tensor)
        train_pred = torch.argmax(train_output, dim=1)
        train_accuracy = accuracy_score(y_train_tensor.numpy(), train_pred.numpy())
        train_loss = criterion(train_output, y_train_tensor).item()
        #akurasi pelatihan
        test_output = model(X_test_tensor)
        test_pred = torch.argmax(test_output, dim=1)
        test_accuracy = accuracy_score(y_test_tensor.numpy(), test_pred.numpy())
        test_loss = criterion(test_output, y_test_tensor).item()
        #akurasi pengujian

    return train_accuracy, train_loss, test_accuracy, test_loss
    #mengembalikan hasil

#fungsi untuk melatih dan mengevaluasi model

In [8]:
hidden_layers_options = [1, 2, 3]
hidden_neurons_options = [8, 16, 32]
activation_functions = ['linear', 'Sigmoid', 'ReLU', 'Softmax', 'Tanh']
epochs_options = [10, 25, 50, 100]
learning_rates = [0.1, 0.01, 0.001, 0.0001]
batch_sizes = [64, 128, 256, 512]
#daftar hyperparameter untuk eksperimen

results = []  # Menyimpan hasil eksperimen
best_result = None
worst_result = None
#melakukan eksperimen dengan semua kombinasi hyperparameter

for hidden_layers in hidden_layers_options:
    for hidden_neurons in hidden_neurons_options:
        for activation_function in activation_functions:
            for epochs in epochs_options:
                for learning_rate in learning_rates:
                    for batch_size in batch_sizes:
                        print(f"Melatih dengan {hidden_layers} lapisan tersembunyi, {hidden_neurons} neuron, fungsi aktivasi {activation_function}, "
                              f"{epochs} epoch, tingkat pembelajaran {learning_rate}, ukuran batch {batch_size}")
                        #mencetak konfigurasi hyperparameter yang sedang digunakan

                        model = create_mlp(X_train_tensor.shape[1], hidden_layers, hidden_neurons, activation_function)
                        #membuat model dengan kombinasi hyperparameter tertentu

                        train_acc, train_loss, test_acc, test_loss = train_and_evaluate(
                            model, X_train_tensor, y_train_tensor, X_test_tensor, y_test_tensor, batch_size, epochs, learning_rate
                        )
                        #melatih dan mengevaluasi model

                        results.append({
                            'Hidden Layers': hidden_layers,
                            'Hidden Neurons': hidden_neurons,
                            'Activation Function': activation_function,
                            'Epochs': epochs,
                            'Learning Rate': learning_rate,
                            'Batch Size': batch_size,
                            'Train Accuracy': train_acc,
                            'Train Loss': train_loss,
                            'Test Accuracy': test_acc,
                            'Test Loss': test_loss
                        })
                        #menyimpan hasil eksperimen

                        print(f"Akurasi Pelatihan: {train_acc * 100:.2f}%")
                        print(f"Loss Pelatihan: {train_loss:.4f}")
                        print(f"Akurasi Pengujian: {test_acc * 100:.2f}%")
                        print(f"Loss Pengujian: {test_loss:.4f}\n")
                        #mencetak hasil akhir dari eksperimen saat ini

                        if best_result is None or test_acc > best_result['Test Accuracy']:
                            best_result = {
                                'Hyperparameters': {
                                    'Hidden Layers': hidden_layers,
                                    'Hidden Neurons': hidden_neurons,
                                    'Activation Function': activation_function,
                                    'Epochs': epochs,
                                    'Learning Rate': learning_rate,
                                    'Batch Size': batch_size
                                },
                                'Test Accuracy': test_acc
                            }
                        if worst_result is None or test_acc < worst_result['Test Accuracy']:
                            worst_result = {
                                'Hyperparameters': {
                                    'Hidden Layers': hidden_layers,
                                    'Hidden Neurons': hidden_neurons,
                                    'Activation Function': activation_function,
                                    'Epochs': epochs,
                                    'Learning Rate': learning_rate,
                                    'Batch Size': batch_size
                                },
                                'Test Accuracy': test_acc
                            }
                            #memperbarui hasil terbaik dan terburuk


Streaming output truncated to the last 5000 lines.
Loss Pengujian: 0.5961

Melatih dengan 3 lapisan tersembunyi, 8 neuron, fungsi aktivasi Sigmoid, 100 epoch, tingkat pembelajaran 0.0001, ukuran batch 512
Akurasi Pelatihan: 58.30%
Loss Pelatihan: 0.6646
Akurasi Pengujian: 62.01%
Loss Pengujian: 0.6592

Melatih dengan 3 lapisan tersembunyi, 8 neuron, fungsi aktivasi ReLU, 10 epoch, tingkat pembelajaran 0.1, ukuran batch 64
Akurasi Pelatihan: 19.11%
Loss Pelatihan: 6.1352
Akurasi Pengujian: 18.18%
Loss Pengujian: 6.2177

Melatih dengan 3 lapisan tersembunyi, 8 neuron, fungsi aktivasi ReLU, 10 epoch, tingkat pembelajaran 0.1, ukuran batch 128
Akurasi Pelatihan: 86.61%
Loss Pelatihan: 0.3352
Akurasi Pengujian: 80.19%
Loss Pengujian: 0.4095

Melatih dengan 3 lapisan tersembunyi, 8 neuron, fungsi aktivasi ReLU, 10 epoch, tingkat pembelajaran 0.1, ukuran batch 256
Akurasi Pelatihan: 86.33%
Loss Pelatihan: 0.3354
Akurasi Pengujian: 78.90%
Loss Pengujian: 0.4362

Melatih dengan 3 lapisan tersem

In [9]:
print("\nBest Hyperparameter Configuration:")
print(f"Hidden Layers: {best_result['Hyperparameters']['Hidden Layers']}")
print(f"Hidden Neurons: {best_result['Hyperparameters']['Hidden Neurons']}")
print(f"Activation Function: {best_result['Hyperparameters']['Activation Function']}")
print(f"Epochs: {best_result['Hyperparameters']['Epochs']}")
print(f"Learning Rate: {best_result['Hyperparameters']['Learning Rate']}")
print(f"Batch Size: {best_result['Hyperparameters']['Batch Size']}")
print(f"Test Accuracy: {best_result['Test Accuracy'] * 100:.2f}%")
#menampilkan best dan worst hyperparameter berdasarkan akurasi


Best Hyperparameter Configuration:
Hidden Layers: 2
Hidden Neurons: 8
Activation Function: Tanh
Epochs: 10
Learning Rate: 0.001
Batch Size: 128
Test Accuracy: 83.44%


In [10]:
print("\nWorst Hyperparameter Configuration:")
print(f"Hidden Layers: {worst_result['Hyperparameters']['Hidden Layers']}")
print(f"Hidden Neurons: {worst_result['Hyperparameters']['Hidden Neurons']}")
print(f"Activation Function: {worst_result['Hyperparameters']['Activation Function']}")
print(f"Epochs: {worst_result['Hyperparameters']['Epochs']}")
print(f"Learning Rate: {worst_result['Hyperparameters']['Learning Rate']}")
print(f"Batch Size: {worst_result['Hyperparameters']['Batch Size']}")
print(f"Test Accuracy: {worst_result['Test Accuracy'] * 100:.2f}%")
#menampilkan hyperparameter terburuk berdasarkan akurasi terendah


Worst Hyperparameter Configuration:
Hidden Layers: 3
Hidden Neurons: 8
Activation Function: ReLU
Epochs: 10
Learning Rate: 0.1
Batch Size: 64
Test Accuracy: 18.18%
